In [397]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pulp import *

## Reading in Dataset

### Fixed Cost of each WTP

In [398]:
fixed_cost = pd.read_excel("../dataset/fixed_cost.xlsx", index_col=0)
fixed_cost.head()

,Low,High
WTP,,
Ampang Intake,6500,9500
Batang Kali,4980,7270
Bernam River Head,6230,9100
Wangsa Maju,3230,4730
Sungai Tengi,2110,6160


### Capacity of each WTP

In [399]:
capacity = pd.read_excel("../dataset/capacity.xlsx", index_col=0)
capacity.head()

,Capacity
WTP,
Ampang Intake,2000000
Batang Kali,2000000
Bernam River Head,2000000
Wangsa Maju,2000000
Sungai Tengi,2000000


### Distribution Loss

In [400]:
distribution_loss = pd.read_excel("../dataset/distribution_loss.xlsx", index_col=0)
distribution_loss

,Ampang Intake,Batang Kali,Bernam River Head,Wangsa Maju,Sungai Tengi
Distribution Loss,,,,,
DMZ1,0.25,0.21,0.28,0.30,0.29
DMZ2,0.27,0.35,0.30,0.26,0.25
DMZ3,0.28,0.29,0.25,0.26,0.31
DMZ4,0.25,0.29,0.27,0.24,0.34
DMZ5,0.28,0.32,0.34,0.23,0.26


### Links between WTP and DMZ

In [401]:
linkage = pd.read_excel("../dataset/linkage.xlsx", index_col=0)
linkage

,Ampang Intake,Batang Kali,Bernam River Head,Wangsa Maju,Sungai Tengi
Links,,,,,
DMZ1,1,1,0,1,1
DMZ2,0,1,0,1,1
DMZ3,1,0,1,1,0
DMZ4,0,0,1,0,1
DMZ5,0,1,0,1,1


### Variable Cost for each combination of (DMZ, WTP)

In [402]:
variable_cost = pd.read_excel("../dataset/variable_costs.xlsx", index_col=0)
variable_cost.head()

,Ampang Intake,Batang Kali,Bernam River Head,Wangsa Maju,Sungai Tengi
Variable Costs,,,,,
DMZ1,12,12,12,12,12
DMZ2,13,13,13,13,13
DMZ3,10,10,10,10,10
DMZ4,8,8,8,8,8
DMZ5,5,5,5,5,5


### Transport Cost for each combination of (DMZ, WTP)

In [403]:
transport_cost = pd.read_excel("../dataset/freight_costs.xlsx", index_col=0)
transport_cost.head()

,Ampang Intake,Batang Kali,Bernam River Head,Wangsa Maju,Sungai Tengi
Freight Costs ($/Container),,,,,
DMZ1,0,12250,1100,16100,8778
DMZ2,13335,0,8617,20244,10073
DMZ3,15400,22750,0,43610,14350
DMZ4,16450,22050,28000,0,29750
DMZ5,13650,15400,24500,29400,0


### Demand for each DMZ

In [404]:
demand = pd.read_excel("../dataset/demand.xlsx", index_col=0)
demand.head()

,Demand
DMZ,
DMZ1,2800000
DMZ2,90000
DMZ3,1700000
DMZ4,145000
DMZ5,160000


In [405]:
demand["Demand"] = pd.to_numeric(demand["Demand"], downcast="float")
demand.head()

,Demand
DMZ,
DMZ1,2800000.0
DMZ2,90000.0
DMZ3,1700000.0
DMZ4,145000.0
DMZ5,160000.0


## Optimisation

In [406]:
# List of all the WTPs
wtp = list(capacity.index)

# List of all the DMZs
dmz = list(demand.index)

# List of (DMZ, WTP) pairs
dmz_wtp_pairs = [(d, w) for d in dmz for w in wtp]

In [407]:
demand.loc[dmz[0], "Demand"]

2800000.0

In [408]:
# Creating the Linear Optimisation Class
model = LpProblem("Optimising water supply", LpMinimize)

C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [409]:
# Creating Decision Variables
output = LpVariable.dicts("Volume", dmz_wtp_pairs, lowBound=0, upBound=None, cat='continuous')

In [410]:
wtp

['Ampang Intake',
 'Batang Kali',
 'Bernam River Head',
 'Wangsa Maju',
 'Sungai Tengi']

In [411]:
# Define the Objective Function
model += \
     lpSum([fixed_cost.loc[w, "High"] * 1000 for w in wtp]) + \
     lpSum([(variable_cost.loc[d, w] + transport_cost.loc[d, w]) * output[(d, w)] for d in dmz for w in wtp])

In [412]:
curr_dmz=dmz[1]
curr_wtp=wtp[0]

In [413]:
linkage.loc[curr_dmz, curr_wtp]

0

In [414]:
1- distribution_loss.loc[curr_dmz, curr_wtp]

0.73

In [415]:
output[(curr_dmz, curr_wtp)] * (1- distribution_loss.loc[curr_dmz, curr_wtp]) * (linkage.loc[curr_dmz, curr_wtp])

0

In [416]:
# type(demand.loc["DMZ1", "Demand"])
# type(1-distribution_loss.loc["DMZ1", wtp[0]])

In [417]:
# Adding Constraints

## Meet demand for each DMZ
for d in dmz:
    model += lpSum([ (output[(d, w)] * (1-distribution_loss.loc[d, w]) * (linkage.loc[d, w])) for w in wtp]) >= demand.loc[d, "Demand"]

## Within the WTP capacity
for w in wtp:
    model += lpSum([output[(d, w)] for d in dmz]) <= capacity.loc[w, "Capacity"]

## No linkage constraint



In [418]:
# Solve the model
model.solve()

1

In [419]:
print("Total Costs = {:,} ($/Month)".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))

Total Costs = 30,821,880,900 ($/Month)

Status: Optimal


## Results

In [420]:
dict_wtp = {}
dict_dmz = {}

In [467]:
# df = pd.DataFrame(0, index=dmz, columns = wtp)
# df

df = pd.DataFrame()
df

""


In [468]:
# Getting the results
for v in model.variables():
    # print(v.name, v.varValue)
    name = v.name.replace("Volume_", "").replace("_", "")
    # print(name)

    combi = eval(name)
    # print(combi[0])

    curr_dmz = combi[0]
    curr_wtp = combi[1]
    volume = v.varValue

    # print("DMZ: ", dmz, " ", "WTP: ", wtp, " ", "Value: ", volume)

    df.loc[curr_dmz, curr_wtp] = volume


In [469]:
# Supply of water from each WTP to each DMZ (including distribution loss)
df

,AmpangIntake,BatangKali,BernamRiverHead,SungaiTengi,WangsaMaju
DMZ1,1722222.20,503584.42,0.0,1564086.80,0.0
DMZ2,0.00,138461.54,0.0,0.00,0.0
DMZ3,277777.78,0.00,2000000.0,0.00,0.0
DMZ4,0.00,0.00,0.0,219696.97,0.0
DMZ5,0.00,0.00,0.0,216216.22,0.0


In [470]:
df.columns = ["Ampang Intake", "Batang Kali", "Bername River Head", "Sungai Tengi", "Wangsa Maju"]
df = df[["Ampang Intake", "Batang Kali", "Bername River Head", "Wangsa Maju", "Sungai Tengi", ]]

In [471]:
# Supply of water from each WTP to each DMZ (including distribution loss)
df

,Ampang Intake,Batang Kali,Bername River Head,Wangsa Maju,Sungai Tengi
DMZ1,1722222.20,503584.42,0.0,0.0,1564086.80
DMZ2,0.00,138461.54,0.0,0.0,0.00
DMZ3,277777.78,0.00,2000000.0,0.0,0.00
DMZ4,0.00,0.00,0.0,0.0,219696.97
DMZ5,0.00,0.00,0.0,0.0,216216.22


In [448]:
distribution_loss

,Ampang Intake,Batang Kali,Bernam River Head,Wangsa Maju,Sungai Tengi
Distribution Loss,,,,,
DMZ1,0.25,0.21,0.28,0.30,0.29
DMZ2,0.27,0.35,0.30,0.26,0.25
DMZ3,0.28,0.29,0.25,0.26,0.31
DMZ4,0.25,0.29,0.27,0.24,0.34
DMZ5,0.28,0.32,0.34,0.23,0.26


In [449]:
useful_amount = 1- distribution_loss.values

In [450]:
temp = df.values * useful_amount
temp

array([[1291666.65  ,  397831.6918,       0.    ,       0.    ,
        1110501.628 ],
       [      0.    ,   90000.001 ,       0.    ,       0.    ,
              0.    ],
       [ 200000.0016,       0.    , 1500000.    ,       0.    ,
              0.    ],
       [      0.    ,       0.    ,       0.    ,       0.    ,
         145000.0002],
       [      0.    ,       0.    ,       0.    ,       0.    ,
         160000.0028]])

In [451]:
temp.shape

(5, 5)

In [459]:
# Supply of water from each WTP to each DMZ, after accounting for water loss
final_df = pd.DataFrame(temp, columns = wtp, index=dmz)
final_df.round(3)

,Ampang Intake,Batang Kali,Bernam River Head,Wangsa Maju,Sungai Tengi
DMZ1,1291666.650,397831.692,0.0,0.0,1110501.628
DMZ2,0.000,90000.001,0.0,0.0,0.000
DMZ3,200000.002,0.000,1500000.0,0.0,0.000
DMZ4,0.000,0.000,0.0,0.0,145000.000
DMZ5,0.000,0.000,0.0,0.0,160000.003


In [432]:
wtp

['Ampang Intake',
 'Batang Kali',
 'Bernam River Head',
 'Wangsa Maju',
 'Sungai Tengi']

In [460]:
supply = final_df.sum(axis=1)
supply

DMZ1    2.800000e+06
DMZ2    9.000000e+04
DMZ3    1.700000e+06
DMZ4    1.450000e+05
DMZ5    1.600000e+05
dtype: float64

In [461]:
for curr_dmz in dmz:
    print(supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
    # print(supply[curr_dmz] >= demand.loc[curr_dmz, "Demand"])
    a = supply[curr_dmz]
    b = demand.loc[curr_dmz, "Demand"]
    print(np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False))

2799999.9698 >= 2800000.0 ?
True
90000.001 >= 90000.0 ?
True
1700000.0016 >= 1700000.0 ?
True
145000.00019999998 >= 145000.0 ?
True
160000.0028 >= 160000.0 ?
True


In [462]:
final_df.sum(axis=0)

Ampang Intake        1.491667e+06
Batang Kali          4.878317e+05
Bernam River Head    1.500000e+06
Wangsa Maju          0.000000e+00
Sungai Tengi         1.415502e+06
dtype: float64

In [430]:
demand.loc["DMZ5", "Demand"]

160000.0